In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
df = pd.read_csv("../data/d10800_BS1_JoinedTrans.csv")
df.head()

,src_file,row_id,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,13,0,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,19,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,21,0,0,0,0,0,3,0,0,...,0,0,0,0,3,0,0,3,0,0
4,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,26,0,0,0,0,0,3,0,0,...,3,0,0,0,0,0,0,0,0,0


In [3]:
X = df.drop(columns=['src_file', 'row_id'])
X.head()

,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,hg38_LY96,hg38_UBE2E2-AS1,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,3,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,3,0,0,3,0,0
4,0,0,0,0,0,3,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


In [4]:
X = X.apply(pd.to_numeric, downcast='integer')

In [5]:
X.shape

(13183, 10800)

In [6]:
y = df['src_file']

In [7]:
from sklearn.feature_selection import VarianceThreshold
from scipy.sparse import csr_matrix

In [8]:
X_sparse = csr_matrix(X)
X_sparse.shape

(13183, 10800)

In [9]:
selector = VarianceThreshold(threshold=0.05)
X_selected = selector.fit_transform(X_sparse)
X_selected

<13183x8123 sparse matrix of type '<class 'numpy.int8'>'
	with 11411028 stored elements in Compressed Sparse Row format>

In [10]:
X = pd.DataFrame(X_selected.todense())
X.head()

,0,1,2,3,4,5,6,7,8,9,...,8113,8114,8115,8116,8117,8118,8119,8120,8121,8122
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,3,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,2,0,0,2,3
3,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,3,0,0,3,0
4,0,0,0,0,0,3,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
def get_groups_and_subgroups(original_array):
    groups = pd.array([s[5] for s in original_array])
    substring_set = {'ALL', 'Bmemory', 'Bnaive', 'CD14+', 'CD16+', 'NonClassicalT', 'T4eff.mem', 'T4naive', 'T8eff.mem', 'T8naive', 'aTreg', 'nonT', 'rTreg'}
    subgroups = []
    for i in range(original_array.size):
       for s in substring_set:
           if s in original_array[i]:
               subgroups.append(s) 
    return groups, pd.array(subgroups)


In [13]:
groups, subgroups = get_groups_and_subgroups(y)

In [14]:
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y

,Group,Subgroups
0,B,Bmemory
1,B,Bmemory
2,B,Bmemory
3,B,Bmemory
4,B,Bmemory
...,...,...
13178,T,T8naive
13179,T,T8naive
13180,T,T8naive
13181,T,T8naive


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=0)

In [16]:
from joblib import dump
dump((X_train, X_test, y_train, y_test), 'preprocessed_data_BS1.joblib')

['preprocessed_data_BS1.joblib']

In [17]:
def load_data(data):
    df = pd.read_csv(data)
    X = df.drop(columns=['src_file', 'row_id'])
    X = X.apply(pd.to_numeric, downcast='integer')
    y = df['src_file']
    X_sparse = csr_matrix(X)
    selector = VarianceThreshold(threshold=0.05)
    X_selected = selector.fit_transform(X_sparse)
    X = pd.DataFrame(X_selected.todense())
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y

In [18]:
X, y = load_data("../data/d10800_BS2_JoinedTrans.csv")
groups, subgroups = get_groups_and_subgroups(y)
dump((X, groups, subgroups), 'preprocessed_data_BS2.joblib')

['preprocessed_data_BS2.joblib']

In [19]:
X, y = load_data("../data/d10800_10x_JoinedTrans.csv")
groups, subgroups = get_groups_and_subgroups(y)
dump((X, groups, subgroups), 'preprocessed_data_10x.joblib')

['preprocessed_data_10x.joblib']

In [20]:
X, y = load_data("../data/d10800_GEO_JoinedTrans.csv")
groups, subgroups = get_groups_and_subgroups(y)
dump((X, groups, subgroups), 'preprocessed_data_GEO.joblib')

['preprocessed_data_GEO.joblib']